In [2]:
%load_ext autoreload
%autoreload 2

# World of Warcraft Log Toy

This notebook toys with data generated from World of Warcraft (WoW) logs.

4 helper classes were created:
- **Record**
  - *Parses and stores an entry of the log*
  - Each line may have differente structure so more classes deriving from this one may be necessary.
  - Each record also represent an in game event.
- **Encounter**
  - *A set of records that represents a fight agains a boss in the game*
  - Each starts with a ENCOUNTER_START event and go until an ENCOUNTER_END event is found.
- **Query**
  - *An object to help create a chain of iterators to build queries (using filters and map like functions)*
- **Predicate**
  - *A Collection of functions that will be applied to those iterators*


In [3]:
from wow.query import Query, Predicate
from wow.log import Record, Encounter

In [236]:
file = r'wow/WoWCombatLog-012622_215632.txt'

with open(file, 'r', encoding='utf-8') as filePtr:
  raw = filePtr.readlines()

In [237]:
log = Query(enumerate(raw)).map(lambda x: Record(x)).list()

In [238]:
qLog = Query(log)
beg = qLog.filter(Predicate.isEncounterStart()).list()
end = qLog.filter(Predicate.isEncounterEnd()).list()

encounters = Query(zip(beg, end)).map(lambda x: Encounter(log, x[0], x[1]))

e = encounters.filter(
    lambda x: (x.timestamp_end - x.timestamp_begin).total_seconds() > 60
).list()
print(len(encounters.list()), ' => ', len(e))


0  =>  17


In [239]:
# e[0]

In [240]:
# Players List

# players = e[0].q.filter(
#     Predicate.isPlayerAction()
# ).map(
#     Predicate.getActorInfo()
# ).set()
# players

In [241]:
# Hostile NPCs & (some of) Their Actions

# e[0].q.filter(
#     Predicate.isCreatureAction()
# ).filter(
#     Predicate.isActorHostile()
# ).map(
#     [ Predicate.getActor(), Predicate.getEvent()]
# ).set()


In [242]:
# Orb of Torment Actions

# orb = e[0].q.filter(
#     Predicate.isActor('"Orb of Torment"')
# ).map(
#     (
#         Predicate.getAction(),
#         Predicate.getEvent(),
#         Predicate.getTarget()

#     )
# ).list()
# orb


In [244]:
encounter_number = 3

orbDisposed = e[encounter_number].q.filter(
    Predicate.isEvent('SPELL_AURA_APPLIED')
).filter(
    Predicate.isAction('"Sorrowful Procession"')
).map(
    (
        Predicate.getTimestamp(),
        Predicate.getActorId(),
        Predicate.getTarget()
    )
).list()

orbBorn = e[encounter_number].q.filter(
    Predicate.isEvent('SPELL_AURA_APPLIED')
).filter(
    Predicate.isAction('"Eternal Torment"')
).map(
    (
        Predicate.getTimestamp(),
        Predicate.getTargetId(),
    )
).list()


In [245]:
import pandas as pd
oDisp = pd.DataFrame(orbDisposed, columns=['TimeStamp', 'ID', 'Player'])
oBorn = pd.DataFrame(orbBorn, columns=['TimeStamp', 'ID'])

orbsData = pd.merge(
    oBorn,
    pd.merge(
        oDisp.groupby('ID')['TimeStamp'].max(),
        oDisp,
        on='TimeStamp',
        how='inner'
    ),
    on='ID',
    how='inner'
)

orbsData['TimeStamp_x'] = pd.to_datetime(
    orbsData['TimeStamp_x'], format="%m/%d %H:%M:%S.%f")

orbsData['TimeStamp_y'] = pd.to_datetime(
    orbsData['TimeStamp_y'], format="%m/%d %H:%M:%S.%f")

pd.merge(
    orbsData,
    pd.DataFrame(orbsData['TimeStamp_y'] - orbsData['TimeStamp_x'], columns=['Time Elapsed']),
    on=orbsData.index
)


,key_0,TimeStamp_x,ID,TimeStamp_y,Player,Time Elapsed
0,0,1900-01-26 22:42:42.729,Creature-0-4220-2450-18865-177117-000071F892,1900-01-26 22:43:02.087,"""Maulfurionx-Azralon""",0 days 00:00:19.358000
1,1,1900-01-26 22:42:44.249,Creature-0-4220-2450-18865-177117-000071F894,1900-01-26 22:42:59.973,"""Djøw-Azralon""",0 days 00:00:15.724000
2,2,1900-01-26 22:43:26.503,Creature-0-4220-2450-18865-177117-000071F8BE,1900-01-26 22:43:49.522,"""Bablowboy-Azralon""",0 days 00:00:23.019000
3,3,1900-01-26 22:43:27.946,Creature-0-4220-2450-18865-177117-000071F8BF,1900-01-26 22:44:06.198,"""Bablowboy-Azralon""",0 days 00:00:38.252000
4,4,1900-01-26 22:44:08.963,Creature-0-4220-2450-18865-177117-000071F8E8,1900-01-26 22:44:28.158,"""Bablowboy-Azralon""",0 days 00:00:19.195000
5,5,1900-01-26 22:44:10.419,Creature-0-4220-2450-18865-177117-000071F8EA,1900-01-26 22:44:35.804,"""Spartanovix-Azralon""",0 days 00:00:25.385000
6,6,1900-01-26 22:44:57.520,Creature-0-4220-2450-18865-177117-000071F919,1900-01-26 22:45:15.301,"""Djøw-Azralon""",0 days 00:00:17.781000
7,7,1900-01-26 22:44:58.994,Creature-0-4220-2450-18865-177117-000071F91A,1900-01-26 22:45:14.816,"""Bablowboy-Azralon""",0 days 00:00:15.822000
8,8,1900-01-26 22:45:00.494,Creature-0-4220-2450-18865-177117-000071F91C,1900-01-26 22:45:18.676,"""Spartanovix-Azralon""",0 days 00:00:18.182000


In [247]:
oDisp

,TimeStamp,ID,Player
0,1/26 22:42:57.182,Creature-0-4220-2450-18865-177117-000071F892,"""Dii-Azralon"""
1,1/26 22:42:59.973,Creature-0-4220-2450-18865-177117-000071F894,"""Djøw-Azralon"""
2,1/26 22:43:02.087,Creature-0-4220-2450-18865-177117-000071F892,"""Maulfurionx-Azralon"""
3,1/26 22:43:45.681,Creature-0-4220-2450-18865-177117-000071F8BE,"""Dii-Azralon"""
4,1/26 22:43:49.522,Creature-0-4220-2450-18865-177117-000071F8BE,"""Bablowboy-Azralon"""
5,1/26 22:43:50.248,Creature-0-4220-2450-18865-177117-000071F8BF,"""Djøw-Azralon"""
6,1/26 22:44:06.198,Creature-0-4220-2450-18865-177117-000071F8BF,"""Bablowboy-Azralon"""
7,1/26 22:44:22.510,Creature-0-4220-2450-18865-177117-000071F8EA,"""Dii-Azralon"""
8,1/26 22:44:28.158,Creature-0-4220-2450-18865-177117-000071F8E8,"""Bablowboy-Azralon"""
9,1/26 22:44:35.804,Creature-0-4220-2450-18865-177117-000071F8EA,"""Spartanovix-Azralon"""
